Created on Tue Feb 14 2023

Genel PnL hesabı

Gain Enerji 

@author: Berkay Akpınar


## Libraries

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import datetime
from UliPlot.XLSX import auto_adjust_xlsx_column_width
from dateutil import relativedelta

## Data Paths

In [3]:
data_path = "data.xlsx"
dengesizlik_path = "dengesizlik_data.xlsx"
super_grup_path = "super_grup.xlsx"
gip_path = "gip_data.xlsx"

## Read Excel

In [41]:
wind1_data = pd.read_excel("data.xlsx",sheet_name='Wind_1')
wind2_data = pd.read_excel(data_path,sheet_name='Wind_2')
hydro1_data = pd.read_excel(data_path,sheet_name='Hydro_1')
hydro2_data = pd.read_excel(data_path,sheet_name='Hydro_2')
fiyat_data = pd.read_excel(data_path,sheet_name='PTF, SMF')
dengesizlik_data = pd.read_excel(dengesizlik_path)
gip_data = pd.read_excel(gip_path)
super_grup_data = pd.read_excel(super_grup_path)



#### Empty Frame

In [49]:
empty_frame = wind1_data.copy()
empty_frame.drop("Gün Öncesi Üretim Tahmini (MWh)", inplace=True, axis=1)
empty_frame.drop("Gün İçi Üretim Tahmini Revizesi (MWh)", inplace=True, axis=1)
empty_frame.drop("Gerçekleşen Üretim  (MWh)", inplace=True, axis=1)
dönem = pd.Series(pd.DatetimeIndex(empty_frame["Tarih"]).month)
empty_frame.assign(Dönem = dönem)

,Tarih,Saat,Dönem
0,2022-01-01,0,1
1,2022-01-01,1,1
2,2022-01-01,2,1
3,2022-01-01,3,1
4,2022-01-01,4,1
...,...,...,...
8755,2022-12-31,19,12
8756,2022-12-31,20,12
8757,2022-12-31,21,12
8758,2022-12-31,22,12


## Variables

In [5]:
wind1_fee_var = 3/100
wind2_fee_var = 3/100
hydro1_fee_var = 2/100
hydro2_fee_var = 1.5/100

wind1_fee_const = -35
wind2_fee_const = -15
hydro1_fee_const = -30
hydro2_fee_const = 0

## Functions

### Max Fiyat Bulan Fonksiyon

In [6]:
def fiyat_max(row):
    if row['Piyasa Takas Fiyatı (TL/MWh)'] > row['Sistem Marjinal Fiyatı (TL/MWh)']:
        return row['Piyasa Takas Fiyatı (TL/MWh)']
    elif row['Piyasa Takas Fiyatı (TL/MWh)'] < row['Sistem Marjinal Fiyatı (TL/MWh)']:
        return row['Sistem Marjinal Fiyatı (TL/MWh)']
    else:
        return row['Piyasa Takas Fiyatı (TL/MWh)']


In [7]:
max_fiyat = fiyat_data.apply(fiyat_max, axis=1)

### Min Fiyat Bulan Fonksiyon

In [8]:
def fiyat_min(row):
    if row['Piyasa Takas Fiyatı (TL/MWh)'] > row['Sistem Marjinal Fiyatı (TL/MWh)']:
        return row['Sistem Marjinal Fiyatı (TL/MWh)']
    elif row['Piyasa Takas Fiyatı (TL/MWh)'] < row['Sistem Marjinal Fiyatı (TL/MWh)']:
        return row['Piyasa Takas Fiyatı (TL/MWh)']
    else:
        return row['Piyasa Takas Fiyatı (TL/MWh)']

In [9]:
min_fiyat = fiyat_data.apply(fiyat_min, axis=1)

### Dengesizlik Fiyatlarını Bulan Fonksiyon

In [10]:
pdf = min_fiyat*0.97
ndf = max_fiyat*1.03

### Dengesizlik Hesaplayan Fonksiyon

In [11]:
def dengesizlik_hesapla(data_set):
    nihai_tahmin = data_set[["Gün Öncesi Üretim Tahmini (MWh)","Gün İçi Üretim Tahmini Revizesi (MWh)"]].sum(axis=1)
    dengesizlik = data_set["Gerçekleşen Üretim  (MWh)"] - nihai_tahmin
    return dengesizlik

### Dengesizlik Tutarı Hesaplayan Fonksiyon

In [12]:
def dengesizlik_tutari_hesapla(dengesizlik):
    dengesizlik_tutari = dengesizlik.copy()
    for i in range(len(dengesizlik)):
        if dengesizlik.iloc[i] > 0:
            dengesizlik_tutari[i] = dengesizlik.iloc[i] * pdf.iloc[i]
        elif dengesizlik.iloc[i] < 0:
            dengesizlik_tutari[i] = dengesizlik.iloc[i] * ndf.iloc[i]    
    return dengesizlik_tutari

### Gip Hesaplayan Fonksiyon

In [13]:
def gip_hesapla():
    pozitif_toplam = wind1_dengesizlik.copy()
    negatif_toplam = wind1_dengesizlik.copy()
    gain_dsg = wind1_data["Gün İçi Üretim Tahmini Revizesi (MWh)"] + wind2_data["Gün İçi Üretim Tahmini Revizesi (MWh)"] + hydro1_data["Gün İçi Üretim Tahmini Revizesi (MWh)"] + hydro2_data["Gün İçi Üretim Tahmini Revizesi (MWh)"]
    gain_dsg_abs = abs(wind1_data["Gün İçi Üretim Tahmini Revizesi (MWh)"]) + abs(wind2_data["Gün İçi Üretim Tahmini Revizesi (MWh)"]) + abs(hydro1_data["Gün İçi Üretim Tahmini Revizesi (MWh)"]) + abs(hydro2_data["Gün İçi Üretim Tahmini Revizesi (MWh)"])
    gip = wind1_data.copy()

    for i in range(len(gain_dsg)):
        pozitif_toplam.iloc[i] = 0
        if wind1_data["Gün İçi Üretim Tahmini Revizesi (MWh)"].iloc[i] > 0:
             pozitif_toplam.iloc[i] = wind1_data["Gün İçi Üretim Tahmini Revizesi (MWh)"].iloc[i]
        if wind2_data["Gün İçi Üretim Tahmini Revizesi (MWh)"].iloc[i] > 0:
             pozitif_toplam.iloc[i] += wind2_data["Gün İçi Üretim Tahmini Revizesi (MWh)"].iloc[i]
        if hydro1_data["Gün İçi Üretim Tahmini Revizesi (MWh)"].iloc[i] > 0:
             pozitif_toplam.iloc[i] += hydro1_data["Gün İçi Üretim Tahmini Revizesi (MWh)"].iloc[i]
        if hydro2_data["Gün İçi Üretim Tahmini Revizesi (MWh)"].iloc[i] > 0:
             pozitif_toplam.iloc[i] += hydro2_data["Gün İçi Üretim Tahmini Revizesi (MWh)"].iloc[i]
    for i in range(len(gain_dsg)):
        negatif_toplam.iloc[i] = 0
        if wind1_data["Gün İçi Üretim Tahmini Revizesi (MWh)"].iloc[i] < 0:
             negatif_toplam.iloc[i] = wind1_data["Gün İçi Üretim Tahmini Revizesi (MWh)"].iloc[i]
        if wind2_data["Gün İçi Üretim Tahmini Revizesi (MWh)"].iloc[i] < 0:
             negatif_toplam.iloc[i] += wind2_data["Gün İçi Üretim Tahmini Revizesi (MWh)"].iloc[i]
        if hydro1_data["Gün İçi Üretim Tahmini Revizesi (MWh)"].iloc[i] < 0:
             negatif_toplam.iloc[i] += hydro1_data["Gün İçi Üretim Tahmini Revizesi (MWh)"].iloc[i]
        if hydro2_data["Gün İçi Üretim Tahmini Revizesi (MWh)"].iloc[i] < 0:
             negatif_toplam.iloc[i] += hydro2_data["Gün İçi Üretim Tahmini Revizesi (MWh)"].iloc[i]

    wind1_gip_oran = (abs(wind1_data["Gün İçi Üretim Tahmini Revizesi (MWh)"])/gain_dsg_abs) * 100
    wind2_gip_oran = (abs(wind2_data["Gün İçi Üretim Tahmini Revizesi (MWh)"])/gain_dsg_abs) * 100
    hydro1_gip_oran = (abs(hydro1_data["Gün İçi Üretim Tahmini Revizesi (MWh)"])/gain_dsg_abs) * 100
    hydro2_gip_oran = (abs(hydro2_data["Gün İçi Üretim Tahmini Revizesi (MWh)"])/gain_dsg_abs) * 100

    wind1_gip_pnl = gain_gip["Gain Saatlik GİP PnL"] * wind1_gip_oran/100
    wind2_gip_pnl = gain_gip["Gain Saatlik GİP PnL"] * wind2_gip_oran/100
    hydro1_gip_pnl = gain_gip["Gain Saatlik GİP PnL"] * hydro1_gip_oran/100
    hydro2_gip_pnl = gain_gip["Gain Saatlik GİP PnL"] * hydro2_gip_oran/100
    gain_gip_pnl = wind1_gip_pnl + wind2_gip_pnl + hydro1_gip_pnl + hydro2_gip_pnl

    gip.drop("Gün Öncesi Üretim Tahmini (MWh)", inplace=True, axis=1)
    gip.drop("Gün İçi Üretim Tahmini Revizesi (MWh)", inplace=True, axis=1)
    gip.drop("Gerçekleşen Üretim  (MWh)", inplace=True, axis=1)

    gip["Wind1 Gip Dağılım oranı"] = wind1_gip_oran
    gip["Wind2 Gip Dağılım oranı"] = wind2_gip_oran
    gip["Hydro1 Gip Dağılım oranı"] = hydro1_gip_oran
    gip["Hydro2 Gip Dağılım oranı"] = hydro2_gip_oran
    gip["Wind1 Gip PnL"] = wind1_gip_pnl
    gip["Wind2 Gip PnL"] = wind2_gip_pnl
    gip["Hydro1 Gip PnL"] = hydro1_gip_pnl
    gip["Hydro2 Gip PnL"] = hydro2_gip_pnl
    gip["Gain Gip PnL"] = gain_gip_pnl

    

    
    
    return gip

### Toplam Gelir Hesaplayan Fonksiyon

In [53]:
def toplam_gelir_hesapla(data_set,dengesizlik_tutari,gip):
    nihai_tahmin = data_set[["Gün Öncesi Üretim Tahmini (MWh)","Gün İçi Üretim Tahmini Revizesi (MWh)"]].sum(axis=1)
    toplam_gelir = dengesizlik_tutari+(data_set["Gün Öncesi Üretim Tahmini (MWh)"]*fiyat_data["Piyasa Takas Fiyatı (TL/MWh)"]) + gip
    return toplam_gelir

### Maksimum Gelir Hesaplayan Fonksiyon

In [58]:
def maksimum_kazanc_hesapla(data_set):
    kazanc = data_set["Gerçekleşen Üretim  (MWh)"] * fiyat_data["Piyasa Takas Fiyatı (TL/MWh)"]    
    return kazanc

### Dengesizlik Maliyeti Hesaplayan Fonksiyon

In [16]:
def dengesizlik_maliyeti_hesapla(maksimum_kazanc,toplam_gelir):
    dengesizlik_maliyeti = maksimum_kazanc - toplam_gelir
    return dengesizlik_maliyeti

### Birim Tablosu Hesaplayan Fonksiyon

In [17]:
def birim_tablosu_hesapla(revize_data):
    
    birim_tablosu = pd.DataFrame(columns=['Periyot', 'Birim Dengesizlik Maliyeti',"Birim KUPST", 'Birim Üretim Geliri','Birim Net Kazanç',"Üretim Miktarı",'Net Kazanç','Mevsim'], dtype=float) 
    
    # Toplam Üretim # 
    ocak_üretim = revize_data["Gerçekleşen Üretim  (MWh)"][0:744].sum()
    subat_üretim = revize_data["Gerçekleşen Üretim  (MWh)"][744:1416].sum()
    mart_üretim = revize_data["Gerçekleşen Üretim  (MWh)"][1416:2160].sum()
    nisan_üretim = revize_data["Gerçekleşen Üretim  (MWh)"][2160:2880].sum()
    mayis_üretim = revize_data["Gerçekleşen Üretim  (MWh)"][2880:3624].sum()
    haziran_üretim = revize_data["Gerçekleşen Üretim  (MWh)"][3624:4344].sum()
    temmuz_üretim = revize_data["Gerçekleşen Üretim  (MWh)"][4344:5088].sum()
    agustos_üretim = revize_data["Gerçekleşen Üretim  (MWh)"][5088:5832].sum()
    eylul_üretim = revize_data["Gerçekleşen Üretim  (MWh)"][5832:6552].sum()
    ekim_üretim = revize_data["Gerçekleşen Üretim  (MWh)"][6552:7296].sum()
    kasim_üretim = revize_data["Gerçekleşen Üretim  (MWh)"][7296:8016].sum()
    aralik_üretim = revize_data["Gerçekleşen Üretim  (MWh)"][8016:8760].sum()
    yillik_üretim = revize_data["Gerçekleşen Üretim  (MWh)"][0:8760].sum()
    
    # Birim Dengesizlik Maliyeti #   
    ocak_maliyet = (revize_data["Dengesizlik Maliyeti"][0:744].sum()) / ocak_üretim
    subat_maliyet = (revize_data["Dengesizlik Maliyeti"][744:1416].sum()) / subat_üretim
    mart_maliyet = (revize_data["Dengesizlik Maliyeti"][1416:2160].sum()) / mart_üretim
    nisan_maliyet = (revize_data["Dengesizlik Maliyeti"][2160:2880].sum()) / nisan_üretim
    mayis_maliyet = (revize_data["Dengesizlik Maliyeti"][2880:3624].sum())/ mayis_üretim
    haziran_maliyet = (revize_data["Dengesizlik Maliyeti"][3624:4344].sum())/ haziran_üretim
    temmuz_maliyet = (revize_data["Dengesizlik Maliyeti"][4344:5088].sum())/ temmuz_üretim
    agustos_maliyet = (revize_data["Dengesizlik Maliyeti"][5088:5832].sum())/ agustos_üretim
    eylul_maliyet = (revize_data["Dengesizlik Maliyeti"][5832:6552].sum())/ eylul_üretim
    ekim_maliyet = (revize_data["Dengesizlik Maliyeti"][6552:7296].sum())/ ekim_üretim
    kasim_maliyet = (revize_data["Dengesizlik Maliyeti"][7296:8016].sum())/ kasim_üretim
    aralik_maliyet = (revize_data["Dengesizlik Maliyeti"][8016:8760].sum())/ aralik_üretim
    yillik_maliyet = (revize_data["Dengesizlik Maliyeti"][0:8760].sum())/ yillik_üretim
    
    # Birim Üretim Geliri #
    ocak_gelir = (revize_data["Toplam Gelir Miktarı"][0:744].sum()) / ocak_üretim
    subat_gelir = (revize_data["Toplam Gelir Miktarı"][744:1416].sum()) / subat_üretim
    mart_gelir = (revize_data["Toplam Gelir Miktarı"][1416:2160].sum()) / mart_üretim
    nisan_gelir = (revize_data["Toplam Gelir Miktarı"][2160:2880].sum()) / nisan_üretim
    mayis_gelir = (revize_data["Toplam Gelir Miktarı"][2880:3624].sum())/ mayis_üretim
    haziran_gelir = (revize_data["Toplam Gelir Miktarı"][3624:4344].sum())/ haziran_üretim
    temmuz_gelir = (revize_data["Toplam Gelir Miktarı"][4344:5088].sum())/ temmuz_üretim
    agustos_gelir = (revize_data["Toplam Gelir Miktarı"][5088:5832].sum())/ agustos_üretim
    eylul_gelir = (revize_data["Toplam Gelir Miktarı"][5832:6552].sum())/ eylul_üretim
    ekim_gelir = (revize_data["Toplam Gelir Miktarı"][6552:7296].sum())/ ekim_üretim
    kasim_gelir = (revize_data["Toplam Gelir Miktarı"][7296:8016].sum())/ kasim_üretim
    aralik_gelir = (revize_data["Toplam Gelir Miktarı"][8016:8760].sum())/ aralik_üretim
    yillik_gelir = (revize_data["Toplam Gelir Miktarı"][0:8760].sum())/ yillik_üretim
    
    # Birim KUPST #
    ocak_kupst = (revize_data["KUPST"][0:744].sum()) / ocak_üretim
    subat_kupst = (revize_data["KUPST"][744:1416].sum()) / subat_üretim
    mart_kupst = (revize_data["KUPST"][1416:2160].sum()) / mart_üretim
    nisan_kupst = (revize_data["KUPST"][2160:2880].sum()) / nisan_üretim
    mayis_kupst = (revize_data["KUPST"][2880:3624].sum())/ mayis_üretim
    haziran_kupst = (revize_data["KUPST"][3624:4344].sum())/ haziran_üretim
    temmuz_kupst = (revize_data["KUPST"][4344:5088].sum())/ temmuz_üretim
    agustos_kupst = (revize_data["KUPST"][5088:5832].sum())/ agustos_üretim
    eylul_kupst = (revize_data["KUPST"][5832:6552].sum())/ eylul_üretim
    ekim_kupst = (revize_data["KUPST"][6552:7296].sum())/ ekim_üretim
    kasim_kupst = (revize_data["KUPST"][7296:8016].sum())/ kasim_üretim
    aralik_kupst = (revize_data["KUPST"][8016:8760].sum())/ aralik_üretim
    yillik_kupst = (revize_data["KUPST"][0:8760].sum())/ yillik_üretim
    
    # Birim Kazanç #
    ocak_birim_kazanc = ocak_gelir - ocak_maliyet
    subat_birim_kazanc = subat_gelir - subat_maliyet
    mart_birim_kazanc = mart_gelir - mart_maliyet
    nisan_birim_kazanc = nisan_gelir - nisan_maliyet
    mayis_birim_kazanc = mayis_gelir - mayis_maliyet
    haziran_birim_kazanc = haziran_gelir - haziran_maliyet
    temmuz_birim_kazanc = temmuz_gelir - temmuz_maliyet
    agustos_birim_kazanc = agustos_gelir - agustos_maliyet
    eylul_birim_kazanc = eylul_gelir - eylul_maliyet
    ekim_birim_kazanc = ekim_gelir - ekim_maliyet
    kasim_birim_kazanc = kasim_gelir - kasim_maliyet
    aralik_birim_kazanc = aralik_gelir - aralik_maliyet
    yillik_birim_kazanc = yillik_gelir - yillik_maliyet
    
    # Net Kazanç #
    ocak_kazanc = f'{(ocak_birim_kazanc * ocak_üretim):.2f}'
    subat_kazanc = f'{(subat_birim_kazanc * subat_üretim):.2f}'
    mart_kazanc = f'{(mart_birim_kazanc * mart_üretim):.2f}'
    nisan_kazanc = f'{(nisan_birim_kazanc * nisan_üretim):.2f}'
    mayis_kazanc = f'{(mayis_birim_kazanc * mayis_üretim):.2f}'
    haziran_kazanc = f'{(haziran_birim_kazanc * haziran_üretim):.2f}'
    temmuz_kazanc = f'{(temmuz_birim_kazanc * temmuz_üretim):.2f}'
    agustos_kazanc = f'{(agustos_birim_kazanc * agustos_üretim):.2f}'
    eylul_kazanc = f'{(eylul_birim_kazanc * eylul_üretim):.2f}'
    ekim_kazanc = f'{(ekim_birim_kazanc * ekim_üretim):.2f}'
    kasim_kazanc = f'{(kasim_birim_kazanc * kasim_üretim):.2f}'
    aralik_kazanc = f'{(aralik_birim_kazanc * aralik_üretim):.2f}'
    yillik_kazanc = f'{(yillik_birim_kazanc * yillik_üretim):.2f}'
    
    # Değerlerin tabloya yazdırılması #
    birim_tablosu['Periyot'] = ["Ocak","Şubat","Mart","Nisan","Mayıs","Haziran","Temmuz","Ağustos","Eylül","Ekim","Kasım","Aralık","Yıllık"]
    birim_tablosu['Birim Dengesizlik Maliyeti'] = [ocak_maliyet,subat_maliyet,mart_maliyet,nisan_maliyet,mayis_maliyet,haziran_maliyet,temmuz_maliyet,agustos_maliyet,eylul_maliyet,ekim_maliyet,kasim_maliyet,aralik_maliyet,yillik_maliyet]
    birim_tablosu['Birim KUPST'] = [ocak_kupst,subat_kupst,mart_kupst,nisan_kupst,mayis_kupst,haziran_kupst,temmuz_kupst,agustos_kupst,eylul_kupst,ekim_kupst,kasim_kupst,aralik_kupst,yillik_kupst]
    birim_tablosu['Birim Üretim Geliri'] = [ocak_gelir,subat_gelir,mart_gelir,nisan_gelir,mayis_gelir,haziran_gelir,temmuz_gelir,agustos_gelir,eylul_gelir,ekim_gelir,kasim_gelir,aralik_gelir,yillik_gelir]
    birim_tablosu['Birim Net Kazanç'] = [ocak_birim_kazanc,subat_birim_kazanc,mart_birim_kazanc,nisan_birim_kazanc,mayis_birim_kazanc,haziran_birim_kazanc,temmuz_birim_kazanc,agustos_birim_kazanc,eylul_birim_kazanc,ekim_birim_kazanc,kasim_birim_kazanc,aralik_birim_kazanc,yillik_birim_kazanc]
    birim_tablosu['Üretim Miktarı'] = [ocak_üretim,subat_üretim,mart_üretim,nisan_üretim,mayis_üretim,haziran_üretim,temmuz_üretim,agustos_üretim,eylul_üretim,ekim_üretim,kasim_üretim,aralik_üretim,yillik_üretim]
    birim_tablosu['Net Kazanç'] = [ocak_kazanc,subat_kazanc,mart_kazanc,nisan_kazanc,mayis_kazanc,haziran_kazanc,temmuz_kazanc,agustos_kazanc,eylul_kazanc,ekim_kazanc,kasim_kazanc,aralik_kazanc,yillik_kazanc]
    birim_tablosu['Mevsim'] = ["Kış","Kış","İlkbahar","İlkbahar","İlkbahar","Yaz","Yaz","Yaz","Sonbahar","Sonbahar","Sonbahar","Kış",""]
    
    return birim_tablosu

### KUPST Hesaplayan Fonksiyon

In [18]:
def kupst_hesapla(data_set):
    üretim = data_set["Gerçekleşen Üretim  (MWh)"]
    kgup = data_set[["Gün Öncesi Üretim Tahmini (MWh)","Gün İçi Üretim Tahmini Revizesi (MWh)"]].sum(axis=1)
    sapma = abs(üretim-kgup)
    kupsm = wind1_dengesizlik.copy()

    for i in range(len(sapma)):
        if (sapma.iloc[i] > (kgup.iloc[i]*0.1)):
            kupsm.iloc[i] = abs(üretim.iloc[i] - kgup.iloc[i]) - (kgup.iloc[i]*0.1)
        else:
            kupsm.iloc[i] = 0
    
    kupst = kupsm*max_fiyat*0.03
    
    return kupst

### Tasarruf Hesaplayan Fonksiyon

In [19]:
def tasarruf_hesapla(dengesizlik,dt):  
    dsg = dengesizlik_data["GAIN Mevcut DSG Imb  (MWh)"]
    toplam_edt = dengesizlik.copy()
    dsg_edt = dsg.copy()
    toplam_dengesizlik = dsg + dengesizlik
    
    #DSG EDT Hesapla
    for i in range(len(dsg)):
        if dsg.iloc[i] > 0:
            dsg_edt[i] = dsg.iloc[i] * (pdf.iloc[i]) 
        elif dsg.iloc[i] < 0:
            dsg_edt[i] = dsg.iloc[i] * (ndf.iloc[i])
    
    #Toplam EDT Hesapla
     
    for i in range(len(toplam_dengesizlik)):
        if toplam_dengesizlik.iloc[i] > 0:
            toplam_edt[i] = toplam_dengesizlik.iloc[i] * (pdf.iloc[i]) 
        elif toplam_dengesizlik.iloc[i] < 0:
            toplam_edt[i] = toplam_dengesizlik.iloc[i] * (ndf.iloc[i])
    
    #Tasarrufu Hesapla
    
    tasarruf = toplam_edt-(dsg_edt+dt)
    
    return tasarruf

### DSG Hesaplayan Fonksiyon

In [20]:
def dsg_hesapla(): 
    pozitif_dengesizlik_toplamı = wind1_dengesizlik_tutari.copy()
    negatif_dengesizlik_toplamı = wind1_dengesizlik_tutari.copy()
    gain_dsg_dengesizlik = 0
    gain_dsg_dt = wind1_dengesizlik_tutari.copy()
    super_grup_dm = wind1_dengesizlik_tutari.copy()
    ck_dengesizlik_dm = wind1_dengesizlik_tutari.copy()
    wind1_tasarruf = wind1_dengesizlik.copy()
    wind1_dis_tasarruf = wind1_dengesizlik.copy()
    wind2_dis_tasarruf = wind1_dengesizlik.copy()
    hydro1_dis_tasarruf = wind1_dengesizlik.copy()
    hydro2_dis_tasarruf = wind1_dengesizlik.copy()
    super_grup = super_grup_data["Super Grup Imb  (MWh)"]
    
    gain_dsg_dengesizlik = wind1_dengesizlik + wind2_dengesizlik + hydro1_dengesizlik + hydro2_dengesizlik
    
    for i in range(len(gain_dsg_dengesizlik)):
        if gain_dsg_dengesizlik.iloc[i] > 0:
            gain_dsg_dt[i] = gain_dsg_dengesizlik.iloc[i] * pdf.iloc[i]
        elif gain_dsg_dengesizlik.iloc[i] < 0:
            gain_dsg_dt[i] = gain_dsg_dengesizlik.iloc[i] * ndf.iloc[i]
            
    gain_dsg_dm = gain_dsg_dengesizlik*fiyat_data["Piyasa Takas Fiyatı (TL/MWh)"] - gain_dsg_dt
    
    toplam_maliyet = wind1_dengesizlik_maliyeti+wind2_dengesizlik_maliyeti+hydro1_dengesizlik_maliyeti+hydro2_dengesizlik_maliyeti
    ic_tasarruf = toplam_maliyet - gain_dsg_dm
    for i in range(len(ic_tasarruf)):
        if ic_tasarruf.iloc[i] < 0.1:
            ic_tasarruf[i] = 0
    wind1_tasarruf = (wind1_dengesizlik_maliyeti/toplam_maliyet)*ic_tasarruf
    wind1_tasarruf = round(wind1_tasarruf, 2)
    
    wind2_tasarruf = (wind2_dengesizlik_maliyeti/toplam_maliyet)*ic_tasarruf
    wind2_tasarruf = round(wind2_tasarruf, 2)
    
    hydro1_tasarruf = (hydro1_dengesizlik_maliyeti/toplam_maliyet)*ic_tasarruf
    hydro1_tasarruf = round(hydro1_tasarruf, 2)
    
    hydro2_tasarruf = (hydro2_dengesizlik_maliyeti/toplam_maliyet)*ic_tasarruf
    hydro2_tasarruf = round(hydro2_tasarruf, 2)


    for i in range(len(super_grup)):
        if super_grup.iloc[i] > 0:
            super_grup_dm[i] = (super_grup.iloc[i]*fiyat_data["Piyasa Takas Fiyatı (TL/MWh)"].iloc[i]) - super_grup.iloc[i] * pdf.iloc[i]
        elif super_grup.iloc[i] < 0:
            super_grup_dm[i] = (super_grup.iloc[i]*fiyat_data["Piyasa Takas Fiyatı (TL/MWh)"].iloc[i]) - super_grup.iloc[i] * ndf.iloc[i]
            
    ck_dengesizlik = super_grup + gain_dsg_dengesizlik
    for i in range(len(ck_dengesizlik)):
        if ck_dengesizlik.iloc[i] > 0:
            ck_dengesizlik_dm[i] = (ck_dengesizlik.iloc[i]*fiyat_data["Piyasa Takas Fiyatı (TL/MWh)"].iloc[i]) - ck_dengesizlik.iloc[i] * pdf.iloc[i]
        elif ck_dengesizlik.iloc[i] < 0:
            ck_dengesizlik_dm[i] = (ck_dengesizlik.iloc[i]*fiyat_data["Piyasa Takas Fiyatı (TL/MWh)"].iloc[i]) - ck_dengesizlik.iloc[i] * ndf.iloc[i]
    dis_tasarruf = (super_grup_dm+gain_dsg_dm-ck_dengesizlik_dm)*6/10
    for i in range(len(dis_tasarruf)):
        if dis_tasarruf.iloc[i] < 0.1:
            dis_tasarruf[i] = 0

    for i in range(len(pozitif_dengesizlik_toplamı)):
        pozitif_dengesizlik_toplamı.iloc[i] = 0
        negatif_dengesizlik_toplamı.iloc[i] = 0
        if wind1_dengesizlik.iloc[i] > 0:
            pozitif_dengesizlik_toplamı.iloc[i] = pozitif_dengesizlik_toplamı.iloc[i] + wind1_dengesizlik.iloc[i]
        elif wind1_dengesizlik.iloc[i] < 0:  
            negatif_dengesizlik_toplamı.iloc[i] = negatif_dengesizlik_toplamı.iloc[i] + wind1_dengesizlik.iloc[i]
        if wind2_dengesizlik.iloc[i] > 0:
            pozitif_dengesizlik_toplamı.iloc[i] = pozitif_dengesizlik_toplamı.iloc[i] + wind2_dengesizlik.iloc[i]
        elif wind2_dengesizlik.iloc[i] < 0:  
            negatif_dengesizlik_toplamı.iloc[i] = negatif_dengesizlik_toplamı.iloc[i] + wind2_dengesizlik.iloc[i]
        if hydro1_dengesizlik.iloc[i] > 0:
            pozitif_dengesizlik_toplamı.iloc[i] = pozitif_dengesizlik_toplamı.iloc[i] +hydro1_dengesizlik.iloc[i]
        elif hydro1_dengesizlik.iloc[i] < 0:  
            negatif_dengesizlik_toplamı.iloc[i] = negatif_dengesizlik_toplamı.iloc[i] + hydro1_dengesizlik.iloc[i]
        if hydro2_dengesizlik.iloc[i] > 0:
            pozitif_dengesizlik_toplamı.iloc[i] = pozitif_dengesizlik_toplamı.iloc[i] +hydro2_dengesizlik.iloc[i]
        elif hydro2_dengesizlik.iloc[i] < 0:  
            negatif_dengesizlik_toplamı.iloc[i] = negatif_dengesizlik_toplamı.iloc[i] + hydro2_dengesizlik.iloc[i]
    
    for i in range(len(dis_tasarruf)):
        if dis_tasarruf.iloc[i] > 0:
            if gain_dsg_dengesizlik.iloc[i] > 0 and wind1_dengesizlik.iloc[i] > 0:
                wind1_dis_tasarruf.iloc[i] = (wind1_dengesizlik.iloc[i]/pozitif_dengesizlik_toplamı.iloc[i]) * dis_tasarruf.iloc[i]
            elif gain_dsg_dengesizlik.iloc[i] < 0 and wind1_dengesizlik.iloc[i] < 0: 
                wind1_dis_tasarruf.iloc[i] = (wind1_dengesizlik.iloc[i]/negatif_dengesizlik_toplamı.iloc[i]) * dis_tasarruf.iloc[i]
            else:
                wind1_dis_tasarruf.iloc[i] = 0
                
            if gain_dsg_dengesizlik.iloc[i] > 0 and wind2_dengesizlik.iloc[i] > 0:
                wind2_dis_tasarruf.iloc[i] = (wind2_dengesizlik.iloc[i]/pozitif_dengesizlik_toplamı.iloc[i]) * dis_tasarruf.iloc[i]
            elif gain_dsg_dengesizlik.iloc[i] < 0 and wind2_dengesizlik.iloc[i] < 0: 
                wind2_dis_tasarruf.iloc[i] = (wind2_dengesizlik.iloc[i]/negatif_dengesizlik_toplamı.iloc[i]) * dis_tasarruf.iloc[i]
            else:
                wind2_dis_tasarruf.iloc[i] = 0
                
            if gain_dsg_dengesizlik.iloc[i] > 0 and hydro1_dengesizlik.iloc[i] > 0:
                hydro1_dis_tasarruf.iloc[i] = (hydro1_dengesizlik.iloc[i]/pozitif_dengesizlik_toplamı.iloc[i]) * dis_tasarruf.iloc[i]
            elif gain_dsg_dengesizlik.iloc[i] < 0 and hydro1_dengesizlik.iloc[i] < 0: 
                hydro1_dis_tasarruf.iloc[i] = (hydro1_dengesizlik.iloc[i]/negatif_dengesizlik_toplamı.iloc[i]) * dis_tasarruf.iloc[i]
            else:
                hydro1_dis_tasarruf.iloc[i] = 0
                
            if gain_dsg_dengesizlik.iloc[i] > 0 and hydro2_dengesizlik.iloc[i] > 0:
                hydro2_dis_tasarruf.iloc[i] = (hydro2_dengesizlik.iloc[i]/pozitif_dengesizlik_toplamı.iloc[i]) * dis_tasarruf.iloc[i]
            elif gain_dsg_dengesizlik.iloc[i] < 0 and hydro2_dengesizlik.iloc[i] < 0: 
                hydro2_dis_tasarruf.iloc[i] = (hydro2_dengesizlik.iloc[i]/negatif_dengesizlik_toplamı.iloc[i]) * dis_tasarruf.iloc[i]
            else:
                hydro2_dis_tasarruf.iloc[i] = 0
                 
        else:
            wind1_dis_tasarruf.iloc[i] = wind2_dis_tasarruf.iloc[i] = hydro1_dis_tasarruf.iloc[i] = hydro2_dis_tasarruf.iloc[i] = 0
            
    #DSG Tablosu
    
    dsg = dengesizlik_data.copy()
    dsg = dsg.drop(dsg.columns[2], axis=1)
    dsg["Wind1 Dengesizlik"] = wind1_dengesizlik
    dsg["Wind2 Dengesizlik"] = wind2_dengesizlik
    dsg["Hydro1 Dengesizlik"] = hydro1_dengesizlik
    dsg["Hydro2 Dengesizlik"] = hydro2_dengesizlik
    dsg["Pozitif Dengesizlik Toplamı"] = pozitif_dengesizlik_toplamı
    dsg["Negatif Dengesizlik Toplamı"] = negatif_dengesizlik_toplamı
    dsg["Gain DSG Dengesizlik"] = gain_dsg_dengesizlik
    dsg["Wind1 Dengesizlik Maliyeti"] = wind1_dengesizlik_maliyeti
    dsg["Wind2 Dengesizlik Maliyeti"] = wind2_dengesizlik_maliyeti
    dsg["Hydro1 Dengesizlik Maliyeti"] = hydro1_dengesizlik_maliyeti
    dsg["Hydro2 Dengesizlik Maliyeti"] = hydro2_dengesizlik_maliyeti
    dsg["Gain DSG Dengesizlik Maliyeti"] = gain_dsg_dm
    dsg["Wind1 İç Tasarruf"] = wind1_tasarruf
    dsg["Wind2 İç Tasarruf"] = wind2_tasarruf
    dsg["Hydro1 İç Tasarruf"] = hydro1_tasarruf
    dsg["Hydro2 İç Tasarruf"] = hydro2_tasarruf
    dsg["Toplam İç Tasarruf"] = ic_tasarruf
    dsg["Süper Grup Dengesizlik"] = super_grup
    dsg["CK Dengesizlik"] = ck_dengesizlik
    dsg["Süper Grup Dengesizlik Maliyeti"] = super_grup_dm
    dsg["CK Dengesizlik Maliyeti"] = ck_dengesizlik_dm
    dsg["Wind1 Dış Tasarruf"] = wind1_dis_tasarruf
    dsg["Wind2 Dış Tasarruf"] = wind2_dis_tasarruf
    dsg["Hydro1 Dış Tasarruf"] = hydro1_dis_tasarruf
    dsg["Hydro2 Dış Tasarruf"] = hydro2_dis_tasarruf
    dsg["Toplam Dış Tasarruf"] = dis_tasarruf
    
    return dsg

### Servis Ücreti Hesaplayan Fonksiyon

In [21]:
def service_fee_hesapla(degisken,sabit,üretim):
    #Servis ücreti =  (Değişken * PTF + Sabit + Bayi Payı)*Üretim    
    
    service_fee = ((degisken * fiyat_data["Piyasa Takas Fiyatı (TL/MWh)"]) + sabit) * üretim
    return service_fee

### Finansman Geliri Hesaplayan Fonksiyon

In [22]:
def finansman_geliri_hesapla(data):
    tablo = data[["Tarih","Gün Öncesi Üretim Tahmini (MWh)"]]
    gelir = data["Gün Öncesi Üretim Tahmini (MWh)"] * fiyat_data["Piyasa Takas Fiyatı (TL/MWh)"]
    weekday = wind1_dengesizlik.copy()
    odeme_gunu = wind1_dengesizlik.copy()
    faiz_geliri = wind1_dengesizlik.copy()
    faiz = (20/100)/365
    # deneme = data["Tarih"] + pd.Timedelta(3, unit="D")
    for i in range(len(data["Tarih"])):
        weekday[i] = data["Tarih"][i].weekday()
    
    for i in range(len(data["Tarih"])):
        if weekday[i] < 3 or weekday[i] == 6:
            odeme_gunu[i] = data["Tarih"][i] + pd.Timedelta(2, unit="D")
        elif weekday[i] == 3 or weekday[i] ==4:
            odeme_gunu[i] = data["Tarih"][i] + pd.Timedelta(4, unit="D")
        elif weekday[i] == 5:
            odeme_gunu[i] = data["Tarih"][i] + pd.Timedelta(3, unit="D")
    
    # deneme = data["Tarih"][30] - data["Tarih"][1]
    for i in range(len(data["Tarih"])):
        faiz_geliri[i] = data["Gün Öncesi Üretim Tahmini (MWh)"][i] * (data["Tarih"][i] + relativedelta.relativedelta(months=1, day=20) - odeme_gunu[i]).days * faiz * fiyat_data["Piyasa Takas Fiyatı (TL/MWh)"][i]
        faiz_geliri[i] = round( faiz_geliri[i] , 2)
    return faiz_geliri

### Gain Gip Hesaplayan Fonksiyon

In [23]:
def gain_gip_hesapla():
    gip_alis_miktari = wind1_dengesizlik.copy()
    gip_satis_miktari = wind1_dengesizlik.copy()
    gip_alis_tutari = wind1_dengesizlik.copy()
    gip_satis_tutari = wind1_dengesizlik.copy()
    gain_gip = wind1_data.copy()
    gip_alis_fiyati = fiyat_data["Piyasa Takas Fiyatı (TL/MWh)"]*0.9
    gip_satis_fiyati = fiyat_data["Piyasa Takas Fiyatı (TL/MWh)"]*1.03
    net_gip_miktari = gip_data["Net Saatlik GİP Miktarı"]
    net_gip_fiyati = gip_data["Net Saatlik GİP Fiyatı (TL/MWh)"]
   
    

    for i in range(len(gip_alis_miktari)):
        gip_alis_miktari.iloc[i] = 0
        if wind1_data["Gün İçi Üretim Tahmini Revizesi (MWh)"].iloc[i] > 0:
            gip_alis_miktari.iloc[i] = wind1_data["Gün İçi Üretim Tahmini Revizesi (MWh)"].iloc[i]*-1
        if wind2_data["Gün İçi Üretim Tahmini Revizesi (MWh)"].iloc[i] > 0:
            gip_alis_miktari.iloc[i] += wind2_data["Gün İçi Üretim Tahmini Revizesi (MWh)"].iloc[i]*-1
        if hydro1_data["Gün İçi Üretim Tahmini Revizesi (MWh)"].iloc[i] > 0:
            gip_alis_miktari.iloc[i] += hydro1_data["Gün İçi Üretim Tahmini Revizesi (MWh)"].iloc[i]*-1
        if hydro2_data["Gün İçi Üretim Tahmini Revizesi (MWh)"].iloc[i] > 0:
            gip_alis_miktari.iloc[i] += hydro2_data["Gün İçi Üretim Tahmini Revizesi (MWh)"].iloc[i]*-1

    for i in range(len(gip_satis_miktari)):
        gip_satis_miktari.iloc[i] = 0
        if wind1_data["Gün İçi Üretim Tahmini Revizesi (MWh)"].iloc[i] < 0:
            gip_satis_miktari.iloc[i] = wind1_data["Gün İçi Üretim Tahmini Revizesi (MWh)"].iloc[i]*-1
        if wind2_data["Gün İçi Üretim Tahmini Revizesi (MWh)"].iloc[i] < 0:
            gip_satis_miktari.iloc[i] += wind2_data["Gün İçi Üretim Tahmini Revizesi (MWh)"].iloc[i]*-1
        if hydro1_data["Gün İçi Üretim Tahmini Revizesi (MWh)"].iloc[i] < 0:
            gip_satis_miktari.iloc[i] += hydro1_data["Gün İçi Üretim Tahmini Revizesi (MWh)"].iloc[i]*-1
        if hydro2_data["Gün İçi Üretim Tahmini Revizesi (MWh)"].iloc[i] < 0:
            gip_satis_miktari.iloc[i] += hydro2_data["Gün İçi Üretim Tahmini Revizesi (MWh)"].iloc[i]*-1
    
    gip_alis_tutari = gip_alis_miktari*gip_alis_fiyati
    gip_satis_tutari = gip_satis_miktari*gip_satis_fiyati
    net_gip_tutari = net_gip_fiyati*net_gip_miktari
    net_gip_pnl = net_gip_tutari - (net_gip_miktari*fiyat_data["Piyasa Takas Fiyatı (TL/MWh)"])

    gain_gip.drop("Gün Öncesi Üretim Tahmini (MWh)", inplace=True, axis=1)
    gain_gip.drop("Gün İçi Üretim Tahmini Revizesi (MWh)", inplace=True, axis=1)
    gain_gip.drop("Gerçekleşen Üretim  (MWh)", inplace=True, axis=1)

    gain_gip["Blok GİP Alış Miktarı"] = gip_alis_miktari
    gain_gip["Blok GİP Alış Tutarı"] = gip_alis_tutari
    gain_gip["Blok GİP Alış Fiyatı"] = gip_alis_fiyati
    gain_gip["Blok GİP Satış Miktarı"] = gip_satis_miktari
    gain_gip["Blok GİP Satış Tutarı"] = gip_satis_tutari
    gain_gip["Blok GİP Satış Fiyatı"] = gip_satis_fiyati
    gain_gip["Net Saatlik Gip Miktarı"] = net_gip_miktari
    gain_gip["Net Saatlik Gip Tutarı"] = net_gip_tutari
    gain_gip["Net Saatlik GİP Fiyatı (TL/MWh)"] = net_gip_fiyati
    gain_gip["Gain Saatlik GİP PnL"] = net_gip_pnl
    

    return gain_gip
    
    

### Net PnL Hesaplayan Fonksiyon

In [56]:
def pnl_hesapla(max_kazanc,ic_tasarruf,küpst,finansal_fayda,servis_ücreti,dengesizlik_maliyeti,gip,dış_tasarruf):
    net_pnl = max_kazanc + ic_tasarruf + küpst + finansal_fayda + servis_ücreti + dengesizlik_maliyeti + gip + dış_tasarruf

    return net_pnl

## Calculations

### Dengesizlikler

In [24]:
wind1_dengesizlik = dengesizlik_hesapla(wind1_data)
wind2_dengesizlik = dengesizlik_hesapla(wind2_data)
hydro1_dengesizlik = dengesizlik_hesapla(hydro1_data)
hydro2_dengesizlik = dengesizlik_hesapla(hydro2_data)

#Toplam Demgesizlik Miktarı
gain_dsg = wind1_dengesizlik+wind2_dengesizlik+hydro1_dengesizlik+hydro2_dengesizlik

### Dengesizlik Tutarları

In [25]:
wind1_dengesizlik_tutari = dengesizlik_tutari_hesapla(wind1_dengesizlik)
wind2_dengesizlik_tutari = dengesizlik_tutari_hesapla(wind2_dengesizlik)
hydro1_dengesizlik_tutari = dengesizlik_tutari_hesapla(hydro1_dengesizlik)
hydro2_dengesizlik_tutari = dengesizlik_tutari_hesapla(hydro2_dengesizlik)

### GİP

In [26]:
# wind1_gip = gip_hesapla(wind1_data)
# wind2_gip = gip_hesapla(wind2_data)
# hydro1_gip = gip_hesapla(hydro1_data)
# hydro2_gip = gip_hesapla(hydro2_data)

gain_gip = gain_gip_hesapla()
gip_tablo = gip_hesapla()

### Toplam Gelirler

In [54]:
wind1_toplam_gelir = toplam_gelir_hesapla(wind1_data,wind1_dengesizlik_tutari,gip_tablo["Wind1 Gip PnL"])
wind2_toplam_gelir = toplam_gelir_hesapla(wind2_data,wind2_dengesizlik_tutari,gip_tablo["Wind2 Gip PnL"])
hydro1_toplam_gelir = toplam_gelir_hesapla(hydro1_data,hydro1_dengesizlik_tutari,gip_tablo["Hydro1 Gip PnL"])
hydro2_toplam_gelir = toplam_gelir_hesapla(hydro2_data,hydro2_dengesizlik_tutari,gip_tablo["Hydro2 Gip PnL"])

### Maksimum Gelirler

In [59]:
wind1_max_gelir = maksimum_kazanc_hesapla(wind1_data)
wind2_max_gelir = maksimum_kazanc_hesapla(wind2_data)
hydro1_max_gelir = maksimum_kazanc_hesapla(hydro1_data)
hydro2_max_gelir = maksimum_kazanc_hesapla(hydro2_data)

### Dengesizlik Maliyetleri

In [62]:
wind1_dengesizlik_maliyeti = dengesizlik_maliyeti_hesapla(wind1_max_gelir,wind1_toplam_gelir)
wind2_dengesizlik_maliyeti = dengesizlik_maliyeti_hesapla(wind2_max_gelir,wind2_toplam_gelir)
hydro1_dengesizlik_maliyeti = dengesizlik_maliyeti_hesapla(hydro1_max_gelir,hydro1_toplam_gelir)
hydro2_dengesizlik_maliyeti = dengesizlik_maliyeti_hesapla(hydro2_max_gelir,hydro2_toplam_gelir)

### KUPST

In [65]:
wind1_kupst = kupst_hesapla(wind1_data)
wind2_kupst = kupst_hesapla(wind2_data)
hydro1_kupst = kupst_hesapla(hydro1_data)
hydro2_kupst = kupst_hesapla(hydro2_data)

### Tasarruflar

In [71]:
wind1_tasarruf = tasarruf_hesapla(wind1_dengesizlik,wind1_dengesizlik_tutari)
wind2_tasarruf = tasarruf_hesapla(wind2_dengesizlik,wind2_dengesizlik_tutari)
hydro1_tasarruf = tasarruf_hesapla(hydro1_dengesizlik,hydro1_dengesizlik_tutari)
hydro2_tasarruf = tasarruf_hesapla(hydro2_dengesizlik,hydro2_dengesizlik_tutari)

### DSG

In [63]:
dsg = dsg_hesapla()

### Finansman Gelirleri

In [67]:
wind1_finansman_geliri = finansman_geliri_hesapla(wind1_data)
wind2_finansman_geliri = finansman_geliri_hesapla(wind2_data)
hydro1_finansman_geliri = finansman_geliri_hesapla(hydro1_data)
hydro2_finansman_geliri = finansman_geliri_hesapla(hydro2_data)

### Servis Ücretleri

In [68]:
wind1_service_fee = service_fee_hesapla(wind1_fee_var, wind1_fee_const, wind1_data["Gerçekleşen Üretim  (MWh)"])
wind2_service_fee = service_fee_hesapla(wind2_fee_var, wind2_fee_const, wind2_data["Gerçekleşen Üretim  (MWh)"])
hydro1_service_fee = service_fee_hesapla(hydro1_fee_var, hydro2_fee_const, hydro1_data["Gerçekleşen Üretim  (MWh)"])
hydro2_service_fee = service_fee_hesapla(hydro2_fee_var, hydro2_fee_const, hydro2_data["Gerçekleşen Üretim  (MWh)"])

In [72]:
wind1_data_revize = wind1_data.copy()
wind2_data_revize = wind2_data.copy()
hydro1_data_revize = hydro1_data.copy()
hydro2_data_revize = hydro2_data.copy()

wind1_data_revize["Dengesizlik Tutarı"] = wind1_dengesizlik_tutari
wind2_data_revize["Dengesizlik Tutarı"] = wind2_dengesizlik_tutari
hydro1_data_revize["Dengesizlik Tutarı"] = hydro1_dengesizlik_tutari
hydro2_data_revize["Dengesizlik Tutarı"] = hydro2_dengesizlik_tutari

wind1_data_revize["Dengesizlik Maliyeti"] = wind1_dengesizlik_maliyeti
wind2_data_revize["Dengesizlik Maliyeti"] = wind2_dengesizlik_maliyeti
hydro1_data_revize["Dengesizlik Maliyeti"] = hydro1_dengesizlik_maliyeti
hydro2_data_revize["Dengesizlik Maliyeti"] = hydro2_dengesizlik_maliyeti

wind1_data_revize["KUPST"] = wind1_kupst
wind2_data_revize["KUPST"] = wind2_kupst
hydro1_data_revize["KUPST"] = hydro1_kupst
hydro2_data_revize["KUPST"] = hydro2_kupst

wind1_data_revize["Toplam Gelir Miktarı"] = wind1_toplam_gelir
wind2_data_revize["Toplam Gelir Miktarı"] = wind2_toplam_gelir
hydro1_data_revize["Toplam Gelir Miktarı"] = hydro1_toplam_gelir
hydro2_data_revize["Toplam Gelir Miktarı"] = hydro2_toplam_gelir

wind1_data_revize["DSG Etkisi"] = wind1_tasarruf
wind2_data_revize["DSG Etkisi"] = wind2_tasarruf
hydro1_data_revize["DSG Etkisi"] = hydro1_tasarruf
hydro2_data_revize["DSG Etkisi"] = hydro2_tasarruf

wind1_birim = birim_tablosu_hesapla(wind1_data_revize)
wind2_birim = birim_tablosu_hesapla(wind2_data_revize)
hydro1_birim = birim_tablosu_hesapla(hydro1_data_revize)
hydro2_birim = birim_tablosu_hesapla(hydro2_data_revize)

### Net PNL

In [76]:
wind1_pnl = pnl_hesapla(wind1_max_gelir,dsg["Wind1 İç Tasarruf"],wind1_kupst,wind1_finansman_geliri,wind1_service_fee,wind1_dengesizlik_maliyeti,gip_tablo["Wind1 Gip PnL"],dsg["Wind1 Dış Tasarruf"])
wind2_pnl = pnl_hesapla(wind2_max_gelir,dsg["Wind2 İç Tasarruf"],wind2_kupst,wind1_finansman_geliri,wind2_service_fee,wind2_dengesizlik_maliyeti,gip_tablo["Wind2 Gip PnL"],dsg["Wind2 Dış Tasarruf"])
hydro1_pnl = pnl_hesapla(hydro1_max_gelir,dsg["Hydro1 İç Tasarruf"],hydro1_kupst,hydro1_finansman_geliri,hydro1_service_fee,hydro1_dengesizlik_maliyeti,gip_tablo["Hydro1 Gip PnL"],dsg["Hydro1 Dış Tasarruf"])
hydro2_pnl = pnl_hesapla(hydro2_max_gelir,dsg["Hydro2 İç Tasarruf"],hydro2_kupst,hydro2_finansman_geliri,hydro2_service_fee,hydro2_dengesizlik_maliyeti,gip_tablo["Hydro2 Gip PnL"],dsg["Hydro2 Dış Tasarruf"])

### Net Birim PnL

In [78]:
wind1_net_birim_pnl = wind1_pnl/wind1_data["Gerçekleşen Üretim  (MWh)"]
wind2_net_birim_pnl = wind2_pnl/wind2_data["Gerçekleşen Üretim  (MWh)"]
hydro1_net_birim_pnl = hydro1_pnl/hydro1_data["Gerçekleşen Üretim  (MWh)"]
hydro2_net_birim_pnl = hydro2_pnl/hydro2_data["Gerçekleşen Üretim  (MWh)"]

## Export

### Günlük Özetler

#### Wind 1

In [79]:
wind1_günlük_özet = empty_frame.copy()
wind1_günlük_özet["Net PnL"] = wind1_pnl
wind1_günlük_özet["Net Birim PnL"] = wind1_net_birim_pnl
wind1_günlük_özet["Üretim"] = wind1_data["Gerçekleşen Üretim  (MWh)"]
wind1_günlük_özet["Enerji Dengesizlik Maliyeti"] = wind1_dengesizlik_maliyeti
wind1_günlük_özet["İç Tasarruf"] = dsg["Wind1 İç Tasarruf"]
wind1_günlük_özet["Dış Tsarruf"] = dsg["Wind1 Dış Tasarruf"]
wind1_günlük_özet["Gip PnL"] = gip_tablo["Wind1 Gip PnL"]
wind1_günlük_özet["KÜPST"] = wind1_kupst
wind1_günlük_özet["Hizmet Bedeli"] = wind1_service_fee
wind1_günlük_özet["Finansal Gelir"] = wind1_finansman_geliri


#### Wind 2

In [80]:
wind2_günlük_özet = empty_frame.copy()
wind2_günlük_özet["Net PnL"] = wind2_pnl
wind2_günlük_özet["Net Birim PnL"] = wind2_net_birim_pnl
wind2_günlük_özet["Üretim"] = wind2_data["Gerçekleşen Üretim  (MWh)"]
wind2_günlük_özet["Enerji Dengesizlik Maliyeti"] = wind2_dengesizlik_maliyeti
wind2_günlük_özet["İç Tasarruf"] = dsg["Wind2 İç Tasarruf"]
wind2_günlük_özet["Dış Tsarruf"] = dsg["Wind2 Dış Tasarruf"]
wind2_günlük_özet["Gip PnL"] = gip_tablo["Wind2 Gip PnL"]
wind2_günlük_özet["KÜPST"] = wind2_kupst
wind2_günlük_özet["Hizmet Bedeli"] = wind2_service_fee
wind2_günlük_özet["Finansal Gelir"] = wind2_finansman_geliri

#### Hydro 1

In [81]:
hydro1_günlük_özet = empty_frame.copy()
hydro1_günlük_özet["Net PnL"] = hydro1_pnl
hydro1_günlük_özet["Net Birim PnL"] = hydro1_net_birim_pnl
hydro1_günlük_özet["Üretim"] = hydro1_data["Gerçekleşen Üretim  (MWh)"]
hydro1_günlük_özet["Enerji Dengesizlik Maliyeti"] = hydro1_dengesizlik_maliyeti
hydro1_günlük_özet["İç Tasarruf"] = dsg["Hydro1 İç Tasarruf"]
hydro1_günlük_özet["Dış Tsarruf"] = dsg["Hydro1 Dış Tasarruf"]
hydro1_günlük_özet["Gip PnL"] = gip_tablo["Hydro1 Gip PnL"]
hydro1_günlük_özet["KÜPST"] = hydro1_kupst
hydro1_günlük_özet["Hizmet Bedeli"] = hydro1_service_fee
hydro1_günlük_özet["Finansal Gelir"] = hydro1_finansman_geliri

#### Hydro 2

In [82]:
hydro2_günlük_özet = empty_frame.copy()
hydro2_günlük_özet["Net PnL"] = hydro2_pnl
hydro2_günlük_özet["Net Birim PnL"] = hydro2_net_birim_pnl
hydro2_günlük_özet["Üretim"] = hydro2_data["Gerçekleşen Üretim  (MWh)"]
hydro2_günlük_özet["Enerji Dengesizlik Maliyeti"] = hydro2_dengesizlik_maliyeti
hydro2_günlük_özet["İç Tasarruf"] = dsg["Hydro2 İç Tasarruf"]
hydro2_günlük_özet["Dış Tsarruf"] = dsg["Hydro2 Dış Tasarruf"]
hydro2_günlük_özet["Gip PnL"] = gip_tablo["Hydro2 Gip PnL"]
hydro2_günlük_özet["KÜPST"] = hydro2_kupst
hydro2_günlük_özet["Hizmet Bedeli"] = hydro2_service_fee
hydro2_günlük_özet["Finansal Gelir"] = hydro2_finansman_geliri

### Aylık Özetler

#### Wind1